In [2]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# If no GPU, you can enable it via: Runtime → Change runtime type → GPU

TensorFlow version: 2.19.0
GPU Available: []


In [3]:
# Clone your repository
!git clone https://github.com/loshithan/EstraNet.git
%cd EstraNet

Cloning into 'EstraNet'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 95 (delta 43), reused 88 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 2.92 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/EstraNet


In [4]:
# Install dependencies
!pip install -r requirements.txt

In [5]:
import os
import gdown

# Create data directory
os.makedirs('data', exist_ok=True)

# ASCADf dataset configuration
file_id = "1WNajWT0qFbpqPJiuePS_HeXxsCvUHI5M"
DATASET_PATH = "data/ASCAD.h5"

if not os.path.exists(DATASET_PATH):
    print("📥 Downloading ASCADf dataset from Google Drive...")
    print("   This may take a few minutes (~1.5 GB)\n")
    
    # Download using gdown
    gdown.download(f"https://drive.google.com/uc?id={file_id}", DATASET_PATH, quiet=False)
    
    print("\n✅ Dataset downloaded successfully!")
else:
    print("✅ Dataset already exists")

# Verify dataset
import h5py
with h5py.File(DATASET_PATH, 'r') as f:
    print(f"\n📊 Dataset info:")
    print(f"  Keys: {list(f.keys())}")
    if 'Profiling_traces' in f:
        print(f"  Profiling traces shape: {f['Profiling_traces/traces'].shape}")
    if 'Attack_traces' in f:
        print(f"  Attack traces shape: {f['Attack_traces/traces'].shape}")

📥 Downloading ASCADf dataset from Google Drive...
   This may take a few minutes (~1.5 GB)



Downloading...
From: https://drive.google.com/uc?id=1WNajWT0qFbpqPJiuePS_HeXxsCvUHI5M
To: /content/EstraNet/data/ASCAD.h5
100%|██████████| 46.6M/46.6M [00:00<00:00, 73.0MB/s]


✅ Dataset downloaded successfully!

📊 Dataset info:
  Keys: ['Attack_traces', 'Profiling_traces']
  Profiling traces shape: (50000, 700)
  Attack traces shape: (10000, 700)


In [6]:
# Verify dataset
import h5py
import os

if os.path.exists('data/ASCAD.h5'):
    with h5py.File('data/ASCAD.h5', 'r') as f:
        print("✓ ASCAD.h5 found!")
        print("  Available keys:", list(f.keys()))
        print("  Profiling traces shape:", f['Profiling_traces']['traces'].shape)
        print("  Attack traces shape:", f['Attack_traces']['traces'].shape)
else:
    print("✗ ASCAD.h5 not found. Please upload the dataset.")

✓ ASCAD.h5 found!
  Available keys: ['Attack_traces', 'Profiling_traces']
  Profiling traces shape: (50000, 700)
  Attack traces shape: (10000, 700)


In [6]:
# ============================================================================
# CONTINUE MAMBA TRAINING - Resume from Checkpoint 11
# ============================================================================

print("🔵 Continuing Mamba Model Training")
print("="*70)

# Configuration
CONFIG = {
    'checkpoint_dir': '/content/drive/MyDrive/EstraNet/checkpoints_mamba',
    'result_path': 'results/mamba',
    'warm_start': True,            # ✅ NEW: Resume training
    'checkpoint_idx': 11,          # ✅ NEW: Start from checkpoint 11
    'train_steps': 200000,         # ✅ CHANGED: Was 50000, now 200k
    'save_steps': 10000,           # ✅ CHANGED: Was 5000
    'train_batch_size': 256,
    'eval_batch_size': 32,
    'learning_rate': 0.00025,
    'model_type': 'mamba',
}

import os
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs(CONFIG['result_path'], exist_ok=True)

# Build training command
train_cmd = f"""
python train_trans.py \\
    --data_path=data/ASCAD.h5 \\
    --checkpoint_dir={CONFIG['checkpoint_dir']} \\
    --model_type={CONFIG['model_type']} \\
    --warm_start={CONFIG['warm_start']} \\
    --checkpoint_idx={CONFIG['checkpoint_idx']} \\
    --dataset=ASCAD \\
    --input_length=10000 \\
    --eval_batch_size={CONFIG['eval_batch_size']} \\
    --n_layer=2 \\
    --d_model=128 \\
    --d_inner=256 \\
    --n_head_softmax=8 \\
    --d_head_softmax=16 \\
    --dropout=0.05 \\
    --conv_kernel_size=3 \\
    --n_conv_layer=2 \\
    --pool_size=20 \\
    --beta_hat_2=150 \\
    --model_normalization=preLC \\
    --softmax_attn=True \\
    --do_train=True \\
    --learning_rate={CONFIG['learning_rate']} \\
    --clip=0.25 \\
    --min_lr_ratio=0.004 \\
    --warmup_steps=0 \\
    --train_batch_size={CONFIG['train_batch_size']} \\
    --train_steps={CONFIG['train_steps']} \\
    --iterations=500 \\
    --save_steps={CONFIG['save_steps']} \\
    --result_path={CONFIG['result_path']}
"""

print("🔄 Resuming Mamba training from checkpoint 11...")
print(f"📂 Checkpoint dir: {CONFIG['checkpoint_dir']}")
print(f"🎯 Training from step ~55,000 to {CONFIG['train_steps']:,}")
print(f"⏱️  This will take ~6-8 hours\n")

!{train_cmd}

🔵 Continuing Mamba Model Training
🔄 Resuming Mamba training from checkpoint 11...
📂 Checkpoint dir: /content/drive/MyDrive/EstraNet/checkpoints_mamba
🎯 Training from step ~55,000 to 200,000
⏱️  This will take ~6-8 hours

2026-02-12 13:44:47.725115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770903887.740303    1101 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770903887.744913    1101 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770903887.756564    1101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770903887.756587    1101 com

In [9]:
# View training progress
import pickle
import matplotlib.pyplot as plt

loss_file = f"{CONFIG['checkpoint_dir']}/loss.pkl"

try:
    with open(loss_file, 'rb') as f:
        loss_dict = pickle.load(f)
    
    steps = sorted(loss_dict.keys())
    train_losses = [loss_dict[s]['train_loss'] for s in steps]
    test_losses = [loss_dict[s]['test_loss'] for s in steps]
    
    plt.figure(figsize=(10, 5))
    plt.plot(steps, train_losses, label='Train Loss')
    plt.plot(steps, test_losses, label='Test Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Progress')
    plt.grid(True)
    plt.show()
    
    print(f"Latest step: {steps[-1]}")
    print(f"Train loss: {train_losses[-1]:.4f}")
    print(f"Test loss: {test_losses[-1]:.4f}")
except FileNotFoundError:
    print("Loss file not found yet. Training hasn't started saving checkpoints.")

NameError: name 'CONFIG' is not defined

In [8]:
# Check if Mamba checkpoints exist
import os

# Check Google Drive path
drive_path = '/content/drive/MyDrive/EstraNet/checkpoints_mamba'
if os.path.exists(drive_path):
    print(f"✅ Found Mamba checkpoint directory!")
    files = os.listdir(drive_path)
    print(f"\nCheckpoint files ({len(files)}):")
    for f in sorted(files):
        print(f"  - {f}")
else:
    print(f"❌ Directory not found: {drive_path}")
    print("\nSearching for checkpoints elsewhere...")
    # Check local directory
    if os.path.exists('./checkpoints_mamba'):
        print("Found in local directory: ./checkpoints_mamba")
        print(os.listdir('./checkpoints_mamba'))

❌ Directory not found: /content/drive/MyDrive/EstraNet/checkpoints_mamba

Searching for checkpoints elsewhere...


In [10]:
# ============================================================================
# EVALUATE MAMBA MODEL - Paste this into a Colab cell
# ============================================================================

print("🔍 Evaluating Mamba Model from Google Drive")
print("="*70)

import tensorflow as tf
import numpy as np
import h5py
import os
from mamba_transformer import MambaNet
from evaluation_utils import compute_key_rank
import matplotlib.pyplot as plt

# Load ASCAD dataset
print("\n📥 Loading ASCAD dataset...")
with h5py.File('data/ASCAD.h5', 'r') as f:
    # Load attack traces
    all_traces = f['Attack_traces']['traces'][()]
    traces = all_traces[:, :10000]
    
    # Load labels (the target key byte values)
    labels = f['Attack_traces']['labels'][()]
    
    # Load metadata - it's a structured array with fields
    print("  Checking metadata structure...")
    metadata = f['Attack_traces']['metadata'][()]
    print(f"  Metadata dtype: {metadata.dtype}")
    
    # ASCAD metadata is structured: plaintext, ciphertext, key (each 16 bytes)
    # Extract byte 2 (3rd byte, index 2) of plaintext and key
    plaintexts = metadata['plaintext'][:, 2].astype(np.uint8)
    keys = metadata['key'][:, 2].astype(np.uint8)

print(f"✅ Loaded {len(traces)} attack traces")
print(f"  Plaintexts (byte 2): {plaintexts[:5]} (dtype: {plaintexts.dtype})")
print(f"  Keys (byte 2): {keys[:5]} (dtype: {keys.dtype})")

# Build Mamba model
print("\n🏗️ Building Mamba model...")
model = MambaNet(
    n_layer=2,
    d_model=128,
    d_state=16,
    d_conv=4,
    expand_factor=2,
    d_inner=256,
    d_head_softmax=16,
    n_head_softmax=8,
    dropout=0.05,
    n_classes=256,
    conv_kernel_size=3,
    n_conv_layer=2,
    pool_size=20,
    beta_hat_2=150,
    model_normalization='preLC',
    use_ff=False,
    softmax_attn=True,
    output_attn=False
)

# Build model with dummy input
dummy_input = tf.zeros((1, 10000))
_ = model(dummy_input, training=False)
print(f"✅ Mamba model built. Parameters: {model.count_params():,}")

# Load checkpoint from Google Drive
checkpoint_dir = '/content/drive/MyDrive/EstraNet/checkpoints_mamba/'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)

if checkpoint_path:
    print(f"\n📂 Loading checkpoint: {checkpoint_path}")
    checkpoint = tf.train.Checkpoint(model=model)
    checkpoint.restore(checkpoint_path).expect_partial()
    print("✅ Checkpoint loaded successfully!")
else:
    print(f"❌ No checkpoint found in {checkpoint_dir}")
    print("Available files:")
    print(os.listdir(checkpoint_dir))
    raise FileNotFoundError("Checkpoint not found")

# Run inference
print("\n🚀 Running inference on 10,000 traces...")
traces_flat = traces.astype(np.float32)

predictions = []
batch_size = 50
num_batches = len(traces) // batch_size

for i in range(num_batches):
    start = i * batch_size
    end = start + batch_size
    batch_preds = model(traces_flat[start:end], training=False)[0].numpy()
    predictions.append(batch_preds)
    
    if (i + 1) % 50 == 0:
        print(f"  Processed {i+1}/{num_batches} batches")

predictions = np.vstack(predictions)
print(f"✅ Inference complete. Shape: {predictions.shape}")

# Compute key ranks
print("\n📊 Computing key ranks (100 experiments)...")
key_rank_list = []

for exp in range(100):
    key_ranks = compute_key_rank(predictions, plaintexts, keys)
    key_rank_list.append(key_ranks)
    if (exp + 1) % 10 == 0:
        print(f"  Experiment {exp+1}/100", end='\r')

mean_ranks = np.mean(np.stack(key_rank_list, axis=0), axis=0)
final_rank = mean_ranks[-1]

print("\n")
print("="*70)
print("MAMBA MODEL RESULTS")
print("="*70)
print(f"🏆 Final Mean Rank (at {len(predictions)} traces): {final_rank:.4f}")

if final_rank < 1.0:
    print("🎉 ATTACK SUCCESSFUL! Key recovered!")
else:
    print(f"⚠️  Attack not yet successful. Best rank: {final_rank:.1f}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(mean_ranks, label=f'Mamba (Final Rank: {final_rank:.1f})', linewidth=2)
plt.axhline(y=27, color='r', linestyle='--', label='Transformer (Rank: 27)', alpha=0.7)
plt.xlabel('Number of Traces')
plt.ylabel('Key Rank (Log Scale)')
plt.yscale('log')
plt.title('Mamba vs Transformer Performance on ASCAD')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('mamba_vs_transformer_results.png', dpi=150)
plt.show()

print(f"\n📈 Plot saved to: mamba_vs_transformer_results.png")
print(f"\n{'='*70}")
print("COMPARISON:")
print(f"  Transformer Final Rank: 27")
print(f"  Mamba Final Rank:       {final_rank:.1f}")
if final_rank < 27:
    improvement = ((27 - final_rank) / 27) * 100
    print(f"  🎯 Mamba is {improvement:.1f}% better!")
elif final_rank > 27:
    degradation = ((final_rank - 27) / 27) * 100
    print(f"  ⚠️  Mamba is {degradation:.1f}% worse")
else:
    print(f"  Similar performance")
print(f"{'='*70}")


🔍 Evaluating Mamba Model from Google Drive

📥 Loading ASCAD dataset...
  Checking metadata structure...
  Metadata dtype: [('plaintext', 'u1', (16,)), ('ciphertext', 'u1', (16,)), ('key', 'u1', (16,)), ('masks', 'u1', (16,)), ('desync', '<u4', (1,))]
✅ Loaded 10000 attack traces
  Plaintexts (byte 2): [170 116 174 222  87] (dtype: uint8)
  Keys (byte 2): [224 224 224 224 224] (dtype: uint8)

🏗️ Building Mamba model...
✅ Mamba model built. Parameters: 425,569
❌ No checkpoint found in /content/drive/MyDrive/EstraNet/checkpoints_mamba/
Available files:


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EstraNet/checkpoints_mamba/'

In [7]:
# ============================================================================
# EVALUATE ALL MAMBA CHECKPOINTS - Find the Best One
# ============================================================================
# Paste this into a Colab cell to test all checkpoints and find the best

print("🔍 Evaluating All Mamba Checkpoints")
print("="*70)

import tensorflow as tf
import numpy as np
import h5py
import os
from mamba_transformer import MambaNet
from evaluation_utils import compute_key_rank
import matplotlib.pyplot as plt

# Load ASCAD dataset once
print("\n📥 Loading ASCAD dataset...")
with h5py.File('data/ASCAD.h5', 'r') as f:
    all_traces = f['Attack_traces']['traces'][()]
    traces = all_traces[:, :10000]
    labels = f['Attack_traces']['labels'][()]
    metadata = f['Attack_traces']['metadata'][()]
    plaintexts = metadata['plaintext'][:, 2].astype(np.uint8)
    keys = metadata['key'][:, 2].astype(np.uint8)

print(f"✅ Loaded {len(traces)} attack traces\n")

# Build Mamba model architecture once
print("🏗️ Building Mamba model architecture...")
model = MambaNet(
    n_layer=2, d_model=128, d_state=16, d_conv=4, expand_factor=2,
    d_inner=256, d_head_softmax=16, n_head_softmax=8, dropout=0.05,
    n_classes=256, conv_kernel_size=3, n_conv_layer=2, pool_size=20,
    beta_hat_2=150, model_normalization='preLC', use_ff=False,
    softmax_attn=True, output_attn=False
)

dummy_input = tf.zeros((1, 10000))
_ = model(dummy_input, training=False)
print(f"✅ Model built: {model.count_params():,} parameters\n")

# Checkpoint directory
checkpoint_dir = '/content/drive/MyDrive/EstraNet/checkpoints_mamba/'

# Find all available checkpoints
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.index')]
checkpoint_numbers = sorted([int(f.split('-')[1].replace('.index', '')) 
                            for f in checkpoint_files if 'trans_long-' in f])

print(f"📂 Found {len(checkpoint_numbers)} checkpoints: {checkpoint_numbers}\n")

# Store results
results = []

# Evaluate each checkpoint
for ckpt_num in checkpoint_numbers:
    checkpoint_path = f'{checkpoint_dir}trans_long-{ckpt_num}'
    
    print(f"{'='*70}")
    print(f"Checkpoint {ckpt_num}")
    print(f"{'='*70}")
    
    # Load checkpoint
    try:
        checkpoint = tf.train.Checkpoint(model=model)
        status = checkpoint.restore(checkpoint_path).expect_partial()
        print(f"✅ Loaded checkpoint {ckpt_num}")
    except Exception as e:
        print(f"❌ Failed to load checkpoint {ckpt_num}: {e}")
        continue
    
    # Run inference
    print(f"🚀 Running inference...")
    traces_flat = traces.astype(np.float32)
    predictions = []
    batch_size = 50
    num_batches = len(traces) // batch_size
    
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        batch_preds = model(traces_flat[start:end], training=False)[0].numpy()
        predictions.append(batch_preds)
    
    predictions = np.vstack(predictions)
    
    # Compute key ranks (using fewer experiments for speed)
    print(f"📊 Computing key ranks...")
    key_rank_list = []
    
    for exp in range(50):  # Reduced from 100 for speed
        key_ranks = compute_key_rank(predictions, plaintexts, keys)
        key_rank_list.append(key_ranks)
    
    mean_ranks = np.mean(np.stack(key_rank_list, axis=0), axis=0)
    final_rank = mean_ranks[-1]
    
    results.append({
        'checkpoint': ckpt_num,
        'rank': final_rank,
        'mean_ranks': mean_ranks
    })
    
    print(f"🏆 Rank: {final_rank:.1f}\n")

# Sort results by rank (best first)
results_sorted = sorted(results, key=lambda x: x['rank'])

# Display results table
print("\n" + "="*70)
print("📊 RESULTS SUMMARY - ALL CHECKPOINTS")
print("="*70)
print(f"{'Checkpoint':<12} {'Rank':<10} {'Status':<20}")
print("-"*70)

best_rank = results_sorted[0]['rank']
for r in results_sorted:
    status = "🏆 BEST!" if r['rank'] == best_rank else ""
    if r['rank'] < 27:
        status += " ✅ Better than Transformer!"
    elif r['rank'] < 100:
        status += " 🟡 Good"
    else:
        status += " ❌ Poor"
    
    print(f"trans_long-{r['checkpoint']:<2}  {r['rank']:<10.1f} {status}")

print("="*70)

# Plot comparison
print("\n📈 Generating comparison plot...")
plt.figure(figsize=(14, 6))

# Plot 1: Rank progression for all checkpoints
plt.subplot(1, 2, 1)
for r in results_sorted[:5]:  # Top 5 checkpoints
    label = f"Ckpt-{r['checkpoint']} (Rank: {r['rank']:.0f})"
    if r['checkpoint'] == results_sorted[0]['checkpoint']:
        plt.plot(r['mean_ranks'], label=label, linewidth=3)
    else:
        plt.plot(r['mean_ranks'], label=label, alpha=0.7)

plt.axhline(y=27, color='r', linestyle='--', label='Transformer (Rank: 27)', linewidth=2)
plt.xlabel('Number of Traces')
plt.ylabel('Key Rank (Log Scale)')
plt.yscale('log')
plt.title('Top 5 Mamba Checkpoints vs Transformer')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Final rank by checkpoint number
plt.subplot(1, 2, 2)
checkpoint_nums = [r['checkpoint'] for r in results]
ranks = [r['rank'] for r in results]
colors = ['green' if r < 27 else 'orange' if r < 100 else 'red' for r in ranks]

plt.bar(checkpoint_nums, ranks, color=colors, alpha=0.7)
plt.axhline(y=27, color='r', linestyle='--', label='Transformer', linewidth=2)
plt.xlabel('Checkpoint Number')
plt.ylabel('Final Rank')
plt.title('Rank by Checkpoint')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('all_checkpoints_comparison.png', dpi=150)
plt.show()

print("✅ Plot saved: all_checkpoints_comparison.png")

# Final recommendation
print("\n" + "="*70)
print("🎯 RECOMMENDATION")
print("="*70)
best_ckpt = results_sorted[0]
print(f"✅ Best checkpoint: trans_long-{best_ckpt['checkpoint']}")
print(f"   Final Rank: {best_ckpt['rank']:.1f}")
print(f"\n🔷 Transformer: 431,233 params → Rank 27")
print(f"🔷 Mamba:       425,569 params → Rank {best_ckpt['rank']:.1f}")

if best_ckpt['rank'] < 27:
    improvement = ((27 - best_ckpt['rank']) / 27) * 100
    print(f"\n🎉 MAMBA WINS! {improvement:.1f}% better with {1.3:.1f}% fewer parameters!")
elif best_ckpt['rank'] < 50:
    print(f"\n🟡 Mamba is competitive but needs more tuning")
else:
    print(f"\n❌ Mamba underperforms - consider architecture changes")

print("="*70)


🔍 Evaluating All Mamba Checkpoints
✅ File replaced with fixed version
✅ Checkpoints deleted

⚠️  NOW RESTART RUNTIME: Runtime → Restart runtime
Then run your training again

📥 Loading ASCAD dataset...
✅ Loaded 10000 attack traces

🏗️ Building Mamba model architecture...
✅ Model built: 425,569 parameters



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EstraNet/checkpoints_mamba/'